# Mask creation

This notebook uses the sparse pixel annotations (always in the `annotations.csv` file) 
from the CoralNet website, and creates a **mask** for each image.

The annotations are given in the following order: `row, column, label` -> `height, width, label`.

The image shapes are in the same order: `height, width, channels`.

! quick reminder !

numpy image: H x W x C

torch image: C x H x W

To create the masks, the `labelset.csv` is going to be used.

*I am going to use all the labels provided by the sources (even if they are not in the "verified" labels), 
because the images have been selected to the "confirmed".*

## Import necessary libraries

In [2]:
from PIL import Image
from skimage import io, transform
from sklearn.preprocessing import MinMaxScaler  

import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import torch

## Get data from the chosen source

In [ ]:
# go to the source
os.chdir("/data/jantina/data/CoralNet/used/BHP_GSR_Fiji/other")

# prepare annotations
annotations = pd.read_csv('annotations.csv')
img_names = annotations.iloc[:, 0]
landmarks = annotations.iloc[:, 1:4]

# prepare labels as dictionary
# using numbers from 1 -> N
labels = pd.read_csv('labelset.csv')["Short Code"]
labels.index = labels.index + 1
labels = labels.to_dict()

print(f"[INFO] There are {len(img_names.unique())} images in this source")
print(f"[INFO] There are {len(labels)} labels in this source")

## Create masks for all images in that source

In [ ]:
for img_name in img_names.unique():
    # go to root
    os.chdir("/data/jantina/data/CoralNet/used/Hamad_New")
    # get image
    try:
        image = io.imread(os.path.join('images', img_name))
        # retrieve the annotations for that image
        landmark = landmarks.iloc[img_names[img_names==img_name].index.values]
        landmark = np.asarray(landmark)

        # get image information
        height = image.shape[0]
        width = image.shape[1]
        colors = image.shape[2]

        # create array with coordinates and label code
        classes = np.copy(landmark) 
        for new, old in labels.items():
            classes[landmark == old] = new

        # create mask 
        mask = np.full((height, width), 255)
        for x in classes:
            try:
                mask[x[0], x[1]] = x[2]
            except:
                print(img_name)
                print("These labels are out of bounds", x[0], x[1], x[2])
                continue

        # switch to png to not loose information
        img_name = img_name.split(".JPG")[0]+".png"

        # check the labels contained in the mask
        print(img_name, np.unique(mask))

        # save mask
        os.chdir("/data/jantina/data/CoralNet/used/Hamad_New/masks")
        cv2.imwrite(img_name, mask)

        # load the mask and check the labels again
        # if they don't match there is a problem
        image = cv2.imread(os.path.join(img_name))
        print(img_name, np.unique(image))
    except:
        continue

## Rotating images

Some images might have tilted when being download, thus the annotations won't match. 

The message `These labels are out of bounds` will appear for these images.

These images will need to be rotated first !

In [ ]:
# go to root
os.chdir("/data/jantina/data/CoralNet/used/Hamad_New")

# take a single image
img_name = "ALM2Q14.JPG"
image = io.imread(os.path.join('images', img_name))
print(image.shape)
plt.imshow(image)

# retrieve the annotations for that image
landmark = landmarks.iloc[img_names[img_names==img_name].index.values]
landmark = np.asarray(landmark)

# rotate
img = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
print(img.shape)
plt.imshow(img)

# replace the current image
os.chdir("/data/jantina/data/CoralNet/used/Hamad_New/images")
cv2.imwrite(img_name, img)

## Converting RGBA to RGB

The RGBA color space is a color space that includes an extra channel (alpha channel) for representing the transparency information of an image.

In [ ]:
for img_name in img_names.unique():
    # go to root
    os.chdir("/data/jantina/data/CoralNet/dataset")
    image = Image.open(os.path.join('images', img_name))
    rgb_image = image.convert('RGB')
    # replace the current image
    os.chdir("/data/jantina/data/CoralNet/dataset/images")
    rgb_image.save(img_name)

## Plot a single image with annotations

In [ ]:
# go to root
os.chdir("/data/jantina/data/CoralNet/used/WAPA_RFM")
annotations = pd.read_csv('other/annotations.csv')
img_names = annotations.iloc[:, 0]
landmarks = annotations.iloc[:, 1:4]

# take a single image
img_name = "2021-11-20-ADELUP-A-T4 20.5-B (PB200870).JPG"

# retrieve the annotations for that image
landmarks = annotations.iloc[:, 1:3]
landmarks = landmarks.iloc[img_names[img_names==img_name].index.values]
landmarks = np.asarray(landmarks).astype('float').reshape(-1, 2)

def show_landmarks(image, landmarks):
    """Show image with landmarks"""
    plt.imshow(image)
    plt.scatter(landmarks[:, 1], landmarks[:, 0], s=50, marker='.', c='r')

# show the image with it's annotations
plt.figure()
show_landmarks(io.imread(os.path.join('images', img_name)),landmarks)
plt.axis('off')
plt.show()

In [ ]:
# plot only annotations
plt.figure()
scatter = plt.scatter(landmarks[:, 1], landmarks[:, 0], s=50, marker='.', c='r')
ax = scatter.axes
ax.invert_yaxis()
plt.axis('off')
plt.show()